In [15]:
# 데이터 경로 설정 및 데이터 불러오기
import os
os.chdir('./data')

import pandas as pd
import numpy as np
df = pd.read_csv("Sonar_Mines_Rocks.csv")

In [16]:
# 특징과 라벨 분리
X = df.drop('Y', axis = 1)
Y = df['Y']

In [17]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [18]:
Train_X.shape # 샘플 156개, 특징 60개 => 단순한 모델 필요

(156, 60)

In [19]:
Train_Y.value_counts()

M    87
R    69
Name: Y, dtype: int64

In [20]:
Train_Y.replace({"M":-1, "R":1}, inplace = True)
Test_Y.replace({"M":-1, "R":1}, inplace = True)

In [21]:
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid

#### Case 1. 복잡도 파라미터가 한 개이면서, 단순하고, 우연성이 어느정도 있는 모델(경사 하강법 사용) - Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression as LR

In [23]:
def LR_model_test(C):
    model = LR(C = C, max_iter = 100000, random_state = 10).fit(Train_X, Train_Y) # 가벼운 모델이므로 max_iter를 크게 잡음
    pred_Y = model.predict(Test_X)
    return f1_score(Test_Y, pred_Y)

In [24]:
print("C = 0.1:\t{}".format(LR_model_test(C = 0.1)))
print("C = 1:\t{}".format(LR_model_test(C = 1))) 
print("C = 5:\t{}".format(LR_model_test(C = 5)))

C = 0.1:	0.5581395348837209
C = 1:	0.7407407407407408
C = 5:	0.7169811320754718


- 튜닝 범위: 0.1 < C < 5
- 아직 확정짓기에는 범위가 넓다 => 더 탐색

In [25]:
print("C = 0.5:\t{}".format(LR_model_test(C = 0.5)))
print("C = 2:\t{}".format(LR_model_test(C = 2)))

# 튜닝 범위: 0.1 < C < 1

C = 0.5:	0.6923076923076924
C = 2:	0.7407407407407408


- C=2인 구간을 굳이 안 봐도 돼
- 최종 탐색 구간: 0.1과 1 사이 (0.1과 0.5 사이도 몰라)

In [26]:
# 파라미터 그리드 설정
LR_parameter_grid = ParameterGrid({"C":np.linspace(0.1, 1, 50),
                                  "max_iter":[100000],
                                  "random_state":[10]})

# 파라미터 튜닝 수행 
best_score = -1
for parameter in LR_parameter_grid:
    model = LR(**parameter).fit(Train_X, Train_Y)
    pred_Y = model.predict(Test_X)
    score = f1_score(Test_Y, pred_Y)
    
    if score > best_score:
        best_score = score
        best_parameter = parameter

print(best_parameter, best_score)

{'C': 0.7061224489795919, 'max_iter': 100000, 'random_state': 10} 0.7407407407407408


#### Case 2. 복잡도 파라미터가 두 개이면서, 단순하고, 우연성이 거의 없는 모델 (Decision Tree)

In [27]:
from sklearn.tree import DecisionTreeClassifier as DTC

In [28]:
def DTC_model_test(max_depth, min_samples_leaf):
    model = DTC(max_depth = max_depth, min_samples_leaf = min_samples_leaf).fit(Train_X, Train_Y) 
    pred_Y = model.predict(Test_X)
    return f1_score(Test_Y, pred_Y) 

In [29]:
for max_depth in [3, 6, 9]:
    for min_samples_leaf in [1, 2, 3]:
        score = DTC_model_test(max_depth = max_depth, min_samples_leaf = min_samples_leaf)
        print("{}-{}:{}".format(max_depth, min_samples_leaf, score))

# max depth가 크고(복잡도 증가) min_samples_leaf가 큰 경우(복잡도 감소) 좋은 성능이 나옴을 확인

3-1:0.7083333333333333
3-2:0.7083333333333333
3-3:0.7083333333333333
6-1:0.7843137254901961
6-2:0.7083333333333333
6-3:0.7692307692307692
9-1:0.7307692307692307
9-2:0.8
9-3:0.7547169811320756


In [38]:
# 파라미터 그리드 설정
DTC_parameter_grid = ParameterGrid({"max_depth": np.arange(6, 15),
                                  "min_samples_leaf": np.arange(2, 5)})

# 파라미터 튜닝 수행 
best_score = -1
for parameter in DTC_parameter_grid:
    model = DTC(**parameter).fit(Train_X, Train_Y)
    pred_Y = model.predict(Test_X)
    score = f1_score(Test_Y, pred_Y)
    
    if score > best_score:
        best_score = score
        best_parameter = parameter

print(best_parameter, best_score)

{'max_depth': 8, 'min_samples_leaf': 2} 0.8


#### Case 3. 복잡도 파라미터가 하나이면서, 우연성이 있는 모델 (신경망)

In [30]:
from sklearn.neural_network import MLPClassifier as MLP

In [31]:
def MLP_model_test(hidden_layer_sizes):
    model = MLP(hidden_layer_sizes = hidden_layer_sizes, random_state = 12).fit(Train_X, Train_Y) 
    pred_Y = model.predict(Test_X)
    return f1_score(Test_Y, pred_Y)

In [39]:
# (5, ): 첫 번째 은닉층에 노드 5개
# (3, 3): 첫 번째 은닉층에 노드 3개, 두 번째 은닉층에 노드 3개
# 오른쪽으로 갈 수록 더 복잡한 형태
for hidden_layer_sizes in [(5, ), (10, ), (3, 3), (5, 5), (10, 10)]:    
    score = MLP_model_test(hidden_layer_sizes = hidden_layer_sizes)
    print(hidden_layer_sizes, score)

# max_iter warning 발생 
# (5, 5)에서는 f1-score가 0이 나옴 
#  => 초기값 영향(0을 찍고 올라오도록 초기값이 설정됨
#  => random_state=12는 좋지 않음)으로 보여짐 (근거: 더 단순한 모델과 복잡한 모델에서는 성능이 나왔으므로)
# (5, )와 (10, 10)에서 best_score가 나옴 => 더 복잡한 모델이 필요할지 판단이 필요

C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(5,) 0.7000000000000001
(10,) 0.6923076923076924


C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(3, 3) 0.12903225806451613
(5, 5) 0.0
(10, 10) 0.7843137254901961


C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [40]:
# 파라미터 그리드 설정
MLP_parameter_grid = ParameterGrid({"random_state": [41, 102, 15],
                                  "hidden_layer_sizes": [(4, 4), (10, 10), (5, 5, 5), (10, 10, 10)],
                                   "max_iter":[200, 2000, 20000]})

# 파라미터 튜닝 수행 
best_score = -1
for parameter in MLP_parameter_grid:
    model = MLP(**parameter).fit(Train_X, Train_Y)
    pred_Y = model.predict(Test_X)
    score = f1_score(Test_Y, pred_Y)
    
    if score > best_score:
        best_score = score
        best_parameter = parameter

print(best_parameter, best_score)

C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-pack

{'hidden_layer_sizes': (10, 10), 'max_iter': 2000, 'random_state': 15} 0.9090909090909091


- 신경망을 튜닝할 때는 random_state, hidden_layer_sizes, max_iter를 같이 튜닝해야 함
- 신경망이나 SVR은 하이퍼 파라미터를 잘 튜닝했을 때 정말 좋은 성능을 기대할 수 있는 모델